## Finetuning Chemberta for FLAVOR Dataset Property Prediction Task using LoRA Technique


In [1]:
#Importing Libraries

import evaluate
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,matthews_corrcoef
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from datasets import load_dataset
import datasets
from peft import LoraConfig, get_peft_model
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from transformers import Trainer
from transformers import TrainingArguments, EarlyStoppingCallback
import pandas as pd
import numpy as np 

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the Datasets

In [66]:
train_flav=pd.read_csv('./fart_train.csv')
train_flav.reset_index(drop=True, inplace=True)
val_flav=pd.read_csv('./fart_val.csv')
val_flav.reset_index(drop=True, inplace=True)
test_flav=pd.read_csv('./fart_test.csv')
test_flav.reset_index(drop=True, inplace=True)

In [67]:
train_flav.drop('Unnamed: 0',axis=1, inplace=True)
val_flav.drop('Unnamed: 0',axis=1, inplace=True)
test_flav.drop('Unnamed: 0',axis=1, inplace=True)

## Chemberta Model Sited in FART Paper

In [14]:
model_checkpoint = "seyonec/SMILES_tokenized_PubChem_shard00_160k"  


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=5,
    trust_remote_code=True
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/SMILES_tokenized_PubChem_shard00_160k and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Other Chemberta Models 

## Model 77M MLM model

In [26]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MLM",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 10M MLM model

In [53]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MLM")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-10M-MLM",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 10M MTR Model

In [17]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MTR")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-10M-MTR",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 77M MTR Model

In [68]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MTR",
    num_labels=5,
    trust_remote_code=True
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 5M MTR Model

In [19]:
## Model 5M MTR Model

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-5M-MTR")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-5M-MTR",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-5M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Preparing Dataset

In [69]:
train_dataset = Dataset.from_pandas(train_flav)
val_dataset = Dataset.from_pandas(val_flav)
test_dataset = Dataset.from_pandas(test_flav)


In [70]:
def tokenize_function(examples):

    return tokenizer(examples["Canonicalized SMILES"], padding="max_length", truncation=True, max_length=512)


train_dataset = train_dataset.map(tokenize_function, batched=True)

val_dataset = val_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 2254/2254 [00:00<00:00, 3882.58 examples/s]


In [71]:
label_encoder = LabelEncoder() 

In [72]:

encoded_labels = label_encoder.fit_transform(train_dataset['Canonicalized Taste'])

train_dataset = train_dataset.add_column('label', encoded_labels)

encoded_labels = label_encoder.fit_transform(val_dataset['Canonicalized Taste'])

val_dataset = val_dataset.add_column('label', encoded_labels)

encoded_labels = label_encoder.fit_transform(test_dataset['Canonicalized Taste'])

test_dataset = test_dataset.add_column('label', encoded_labels) 

In [73]:
columns_to_remove = ["Canonicalized SMILES", "Standardized SMILES", 
                     "Canonicalized Taste", "Original Labels", "Source", "is_multiclass"]

# Remove columns from all datasets
train_dataset = train_dataset.remove_columns(columns_to_remove)
val_dataset = val_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove) 

### Compute Metrics


In [32]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    probabilities= softmax(logits, axis=1)
    mcc = matthews_corrcoef(labels, predictions)

        
    return {
            "eval_mcc_metric": mcc,
            "Accuracy": metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities,multi_class="ovr"),  # AUC-ROC requires probabilities
            "Precision": precision_score(labels, predictions,average="macro"),
            "Recall": recall_score(labels, predictions,average="macro"),
            "F1-score": f1_score(labels, predictions,average="macro")
        }

### define LoRA Config

In [11]:

lora_config = LoraConfig(
    task_type="SEQ_CLS",  # Sequence classification task
    r=8,  # Rank of LoRA matrices
    lora_alpha=16,  # Scaling factor double of rank( from the rule of thumb)
    target_modules='all-linear',
    lora_dropout=0.1,  
    #init_lora_weights="gaussian"
)

model = get_peft_model(model, lora_config)

### Define Custom Trainer

In [33]:
#focal loss computation

def focal_loss_multiclass(inputs, targets, alpha=1, gamma=2):
    log_prob = F.log_softmax(inputs, dim=-1)
    prob = torch.exp(log_prob)  # Convert log probabilities back to normal probabilities

    targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
    pt = torch.sum(prob * targets_one_hot, dim=-1)  # Get probability of the true class

    focal_loss = -alpha * (1 - pt) ** gamma * torch.sum(log_prob * targets_one_hot, dim=-1)
    
    return focal_loss.mean() 

In [34]:

class CustomTrainer(Trainer):
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = focal_loss_multiclass(logits, labels)
        
        return (loss, outputs) if return_outputs else loss 

### Train The model

In [14]:

training_args = TrainingArguments(
    output_dir="./results_flavor_chemberta_100m_mtr_lora",
    evaluation_strategy="steps",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="steps",
    logging_dir="./logs_flavor_chemberta",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=10,
    remove_unused_columns=False,
    metric_for_best_model='eval_mcc_metric',
    greater_is_better=True,  
    load_best_model_at_end=True,

         
)

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = CustomTrainer(

    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Hyperparameter tuning using WandB

In [2]:
import wandb
import os

In [23]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [3]:
def data_load():
    train_flav=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/fart_train.csv')
    val_flav=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/fart_val.csv')
    train_flav.drop('Unnamed: 0',axis=1, inplace=True)
    val_flav.drop('Unnamed: 0',axis=1, inplace=True)

    return train_flav, val_flav  

In [4]:
def data_prep(data_process):

    dataset = Dataset.from_pandas(data_process)
       

    return dataset

def tokenize_function(examples,tokenizer):

    return tokenizer(examples["Canonicalized SMILES"], padding="max_length", truncation=True, max_length=512)


In [5]:
def label_encoding(dataset):

    label_encoder = LabelEncoder()

    encoded_labels = label_encoder.fit_transform(dataset['Canonicalized Taste'])

    dataset = dataset.add_column('label', encoded_labels)
    
    columns_to_remove = ["Canonicalized SMILES", "Standardized SMILES", 
                     "Canonicalized Taste", "Original Labels", "Source", "is_multiclass"]


    dataset = dataset.remove_columns(columns_to_remove)

    return dataset

In [6]:
from peft import LoraConfig, get_peft_model

def lora_config(r, lora_alpha, dropout):

    lora_config = LoraConfig(
        task_type="SEQ_CLS",  # Sequence classification task
        r=r,  # Rank of LoRA matrices
        lora_alpha=lora_alpha,  # Scaling factor double of rank( from the rule of thumb)
        target_modules='all-linear',
        lora_dropout=dropout  # Dropout rate
        #init_lora_weights="gaussian"
    )

    return lora_config

In [7]:

def focal_loss_multiclass(inputs, targets, alpha=1, gamma=2):
    log_prob = F.log_softmax(inputs, dim=-1)
    prob = torch.exp(log_prob)  # Convert log probabilities back to normal probabilities

    targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
    pt = torch.sum(prob * targets_one_hot, dim=-1)  # Get probability of the true class

    focal_loss = -alpha * (1 - pt) ** gamma * torch.sum(log_prob * targets_one_hot, dim=-1)
    
    return focal_loss.mean()

In [8]:
class CustomTrainer(Trainer):
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = focal_loss_multiclass(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

## 100M MTR Model hyperparameter tuning

In [ ]:

def run_training():

    run = wandb.init(project="flavor analysis chemberta Hyperparameter Tuning")
    config = run.config


    save_dir = f"./models_Mlm_10_Chemberta/{wandb.run.id}"  # Unique directory for each run
    os.makedirs(save_dir, exist_ok=True)

   
    tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MLM")


    train_data, val_data=data_load()
    training_data=data_prep(train_data)
    validation_data=data_prep(val_data)    
    training_data=training_data.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    validation_data=validation_data.map(lambda x: tokenize_function(x, tokenizer), batched=True)


    training_data=label_encoding(training_data)
    validation_data=label_encoding(validation_data)


    # Load the model with a classification head

    
    model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-10M-MLM",                                 # Define any other model here
    num_labels=5,
    trust_remote_code=True
    )

    peft_config = lora_config(config.r, config.lora_alpha, config.dropout)
    lora_model = get_peft_model(model, peft_config)
    lora_model.print_trainable_parameters()

    training_args = TrainingArguments(
    output_dir=save_dir,
    evaluation_strategy="steps",
    learning_rate=config.lr,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="steps",
    logging_dir="./logs_flavor_chem_wandb",
    logging_strategy="steps",
    logging_steps=100,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="eval_mcc_metric",
    greater_is_better=True,
    remove_unused_columns=False,

    )


    metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):

        logits, labels = eval_pred

        predictions = np.argmax(logits, axis=-1)
        probabilities= softmax(logits, axis=1)
        mcc = matthews_corrcoef(labels, predictions)

            
        return {
                "eval_mcc_metric": mcc,
                "Accuracy": metric.compute(predictions=predictions, references=labels)["accuracy"],
                "AUC-ROC": roc_auc_score(labels, probabilities,multi_class="ovr"),  # AUC-ROC requires probabilities
                "Precision": precision_score(labels, predictions,average="macro"),
                "Recall": recall_score(labels, predictions,average="macro"),
                "F1-score": f1_score(labels, predictions,average="macro")
            }


    trainer_flavor = CustomTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=training_data,
    eval_dataset= validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    trainer_flavor.train()
    trainer_flavor.save_model(save_dir)
        
    print(f"Model saved to {save_dir}")

    wandb.finish()

    

### To check best params for more number of epochs

In [ ]:
trainer_flavor.train()
trainer_flavor.save_model('./manual_config_chemberta')

### Defining Parameter

In [ ]:
def main():

    sweep_config = {
    "name": "Flavor Hyperparameter Tuning",
    "method": "bayes",
    "metric": {
        "goal": "maximize", 
        "name": "eval/mcc_metric"
    },
    "parameters": {
        "lr": {
            "distribution": "uniform",
            "min": 1e-5,  
            "max": 2e-3
        },
        "r": {
            "values": [4,8,16,32,64, 128]
        },
        "lora_alpha": {
            "values": [4,8,16,32,64,128]
        },
        "dropout": {
            "values": [0.0,0.1,0.2]
        },
        
        "optimizer": {
            "value": ["adamw"]
        }
    }
}

    sweep_id = wandb.sweep(sweep_config, project="huggingface")
    wandb.agent(sweep_id, function=run_training, count=5)

    api = wandb.Api()
    sweep = api.sweep(f"huggingface/{sweep_id}")
    print(sweep.runs[0].summary_metrics)

    runs_with_rmse = [run for run in sweep.runs if 'eval/mcc_metric' in run.summary_metrics]
    if runs_with_rmse:
        # Sort by rmse in descending order (maximize)
        best_run = sorted(runs_with_rmse, key=lambda run: run.summary_metrics['eval/mcc_metric'])[0]
    else:
        raise ValueError("No runs found with 'eval/mcc_metric' metric.")

    best_hyperparameters = best_run.config
    print(f"Best hyperparameters: {best_hyperparameters}")
    

if __name__ == "__main__":
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    main()

Create sweep with ID: p5bx31ga
Sweep URL: https://wandb.ai/harodharsha21-iit-ropar/huggingface/sweeps/p5bx31ga


wandb: Agent Starting Run: oggmqthu with config:
wandb: 	dropout: 0.1
wandb: 	lora_alpha: 128
wandb: 	lr: 0.0017052657096512777
wandb: 	optimizer: ['adamw']
wandb: 	r: 128
wandb: Currently logged in as: harodharsha21 (harodharsha21-iit-ropar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Map: 100%|██████████| 2254/2254 [00:00<00:00, 3939.05 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,128,773 || all params: 5,706,234 || trainable%: 37.3061


...

Step,Training Loss,Validation Loss,Mcc Metric,Accuracy,Auc-roc,Precision,Recall,F1-score
100,0.361200,0.218163,0.687245,0.839397,0.942113,0.613976,0.595243,0.601145
200,0.277700,0.183562,0.745702,0.866460,0.959946,0.830733,0.774714,0.793639
300,0.272400,0.177634,0.746863,0.864241,0.961250,0.826519,0.719926,0.738828
400,0.239000,0.196164,0.670627,0.834073,0.958913,0.768313,0.715214,0.735569
500,0.239200,0.176210,0.747892,0.865572,0.956969,0.733378,0.783156,0.754735
600,0.216800,0.158084,0.759298,0.874445,0.967742,0.744523,0.839834,0.760698
700,0.200000,0.135716,0.771171,0.882431,0.975543,0.863401,0.783400,0.815475
800,0.199000,0.139380,0.764518,0.874889,0.974871,0.850055,0.728172,0.753449
900,0.205800,0.132802,0.775513,0.884206,0.974390,0.782033,0.856066,0.807673
1000,0.193200,0.123457,0.771549,0.883319,0.976518,0.870169,0.782056,0.817798


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model saved to ./models_MTR100_Chemberta/to3uvb1h


eval/AUC-ROC,▁▅▅▄▄▆█▇▇█▇▇████
eval/Accuracy,▂▅▅▁▅▆▇▆▇▇▆▇▇▇██
eval/F1-score,▁▇▅▅▆▆▇▆▇▇█▇█▇██
eval/Precision,▁▇▇▅▄▅█▇▆█▆▆▆▆▆▆
eval/Recall,▁▅▄▄▆▇▆▄▇▆█▆█▆██
eval/loss,█▆▅▆▅▄▂▂▂▁▂▂▁▁▁▁
eval/mcc_metric,▂▅▅▁▅▆▇▆▇▇▇▇▇▇██
eval/runtime,▂▃▂█▁▂▁▂▂▂▂▁▁▁▁▁
eval/samples_per_second,▇▆▇▁█▇█▇▇▇▇█████
eval/steps_per_second,▇▆▇▁█▇█▇▇▇▇█████
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇███


{'_runtime': 295.493498149, '_step': 32, '_timestamp': 1742899458.7640474, '_wandb': {'runtime': 295}, 'eval/AUC-ROC': 0.9776794999278964, 'eval/Accuracy': 0.8935226264418811, 'eval/F1-score': 0.8237965481576575, 'eval/Precision': 0.7952690077541764, 'eval/Recall': 0.8734746496338153, 'eval/loss': 0.11860866844654085, 'eval/mcc_metric': 0.7954413436068798, 'eval/runtime': 4.0552, 'eval/samples_per_second': 555.832, 'eval/steps_per_second': 17.508, 'total_flos': 852563953612800, 'train/epoch': 5, 'train/global_step': 1645, 'train/grad_norm': 0.9067480564117432, 'train/learning_rate': 2.9899161842693692e-05, 'train/loss': 0.1448, 'train_loss': 0.20984870858467825, 'train_runtime': 287.0036, 'train_samples_per_second': 183.221, 'train_steps_per_second': 5.732}
Best hyperparameters: {'r': 128, 'lr': 0.0011492784634238222, 'bf16': False, 'fp16': False, 'fsdp': [], 'seed': 42, 'tf32': None, 'debug': [], 'optim': 'adamw_torch', 'top_k': 50, 'top_p': 1, 'is_gpu': True, 'prefix': None, 'do_eval

### Evaluation

In [42]:
import torch
import numpy as np
import random

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# Ensure deterministic behavior in PyTorch computations
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [30]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-10M-MTR",
    num_labels=5,
    problem_type="single_label_classification",    
    trust_remote_code=True,
    
)

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MTR",trust_remote_code=True)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
from peft import PeftModel  

adapter_model = PeftModel.from_pretrained(base_model, "/home/raghvendra2/Molformer_Finetuning/results_flavor_chemberta_wandb_reepoch_10mtr/checkpoint-1000")

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")


In [19]:
#compute metrics
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    probabilities= softmax(logits, axis=1)
    mcc = matthews_corrcoef(labels, predictions)

        
    return {
            "eval_mcc_metric": mcc,
            "Accuracy": metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities,multi_class="ovr"),  # AUC-ROC requires probabilities
            "Precision": precision_score(labels, predictions,average="macro"),
            "Recall": recall_score(labels, predictions,average="macro"),
            "F1-score": f1_score(labels, predictions,average="macro")
        }


In [28]:
training_args = TrainingArguments(
    output_dir="./test_results_flavor",
    per_device_eval_batch_size=32,
    report_to="none",  # Disable logging to W&B for test
    

)

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR",trust_remote_code=True)

In [54]:
trainer= CustomTrainer(
    model=adapter_model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    )

/tmp/ipykernel_3133714/1987706221.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer= CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


### Checking for all 77 MTR Moels

In [17]:
from peft import PeftModel

In [ ]:

import os

models_dir = "./models_MTR100_Chemberta"

for ckpt in os.listdir(models_dir):
    model_path = os.path.join(models_dir, ckpt)
    
    if os.path.isdir(model_path):
        adapter_config_path = os.path.join(model_path, "adapter_config.json")
        
        if not os.path.exists(adapter_config_path):
            print(f"⚠️ Missing 'adapter_config.json' in {model_path}")

# Evaluate each saved model

valid_checkpoints = [
    os.path.join(models_dir, ckpt)
    for ckpt in os.listdir(models_dir)
    if os.path.isdir(os.path.join(models_dir, ckpt)) and 
       os.path.exists(os.path.join(models_dir, ckpt, "adapter_config.json"))
]

print("Valid Model Checkpoints:", valid_checkpoints)

for model_path in valid_checkpoints:
    print(f"Evaluating model: {model_path}")

    adapter_model = PeftModel.from_pretrained(base_model, model_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()
    print(f"Test Results for {model_path}: {test_results_clin}")



⚠️ Missing 'adapter_config.json' in ./models_MTR100_Chemberta/ujye8pur
⚠️ Missing 'adapter_config.json' in ./models_MTR100_Chemberta/5k0m1ybg
⚠️ Missing 'adapter_config.json' in ./models_MTR100_Chemberta/vx2zkfhq
⚠️ Missing 'adapter_config.json' in ./models_MTR100_Chemberta/tl30zdlb
⚠️ Missing 'adapter_config.json' in ./models_MTR100_Chemberta/t6motadp
Valid Model Checkpoints: ['./models_MTR100_Chemberta/glzr8jim', './models_MTR100_Chemberta/y6ioij00', './models_MTR100_Chemberta/imo6nm9x', './models_MTR100_Chemberta/to3uvb1h', './models_MTR100_Chemberta/oggmqthu']
Evaluating model: ./models_MTR100_Chemberta/glzr8jim


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_2954178/2028187298.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer =

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_2954178/2028187298.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer =

Test Results for ./models_MTR100_Chemberta/glzr8jim: {'eval_mcc_metric': 0.7857732041076384, 'eval_loss': 0.13933634757995605, 'eval_model_preparation_time': 0.0047, 'eval_Accuracy': 0.8864241348713399, 'eval_AUC-ROC': 0.9560709660479298, 'eval_Precision': 0.80451841570831, 'eval_Recall': 0.7921601853640271, 'eval_F1-score': 0.7971128505954014, 'eval_runtime': 5.277, 'eval_samples_per_second': 427.137, 'eval_steps_per_second': 13.455}
Evaluating model: ./models_MTR100_Chemberta/y6ioij00


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_2954178/2028187298.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer =

...

Test Results for ./models_MTR100_Chemberta/imo6nm9x: {'eval_mcc_metric': 0.7818169758422309, 'eval_loss': 0.1437530368566513, 'eval_model_preparation_time': 0.0037, 'eval_Accuracy': 0.8850931677018633, 'eval_AUC-ROC': 0.9721605705286832, 'eval_Precision': 0.8250201990356656, 'eval_Recall': 0.7875592578624443, 'eval_F1-score': 0.8045247463831717, 'eval_runtime': 3.9065, 'eval_samples_per_second': 576.986, 'eval_steps_per_second': 18.175}
Evaluating model: ./models_MTR100_Chemberta/to3uvb1h


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_2954178/2028187298.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer =

Test Results for ./models_MTR100_Chemberta/to3uvb1h: {'eval_mcc_metric': 0.777782885687969, 'eval_loss': 0.1434350460767746, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.883318544809228, 'eval_AUC-ROC': 0.9714154098726826, 'eval_Precision': 0.8247383778991484, 'eval_Recall': 0.7833793596443682, 'eval_F1-score': 0.8022087198646425, 'eval_runtime': 4.0312, 'eval_samples_per_second': 559.135, 'eval_steps_per_second': 17.613}
Evaluating model: ./models_MTR100_Chemberta/oggmqthu


Test Results for ./models_MTR100_Chemberta/oggmqthu: {'eval_mcc_metric': 0.7667172633940723, 'eval_loss': 0.14829595386981964, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8757763975155279, 'eval_AUC-ROC': 0.9531141891298681, 'eval_Precision': 0.7546311912638144, 'eval_Recall': 0.746212206833805, 'eval_F1-score': 0.7496316603246458, 'eval_runtime': 4.0222, 'eval_samples_per_second': 560.384, 'eval_steps_per_second': 17.652}


: 

### Checking for check points of glzr8jim model

In [52]:
from peft import PeftModel  

adapter_model = PeftModel.from_pretrained(base_model, "/home/raghvendra2/Molformer_Finetuning/models_MTR100_Chemberta/to3uvb1h/checkpoint-1645")

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")


In [33]:
test_results_flavor = trainer.evaluate()

print("Test Results for checkpoint 1645:", test_results_flavor)

Test Results for checkpoint 1645: {'eval_mcc_metric': 0.7857732041076384, 'eval_loss': 0.13933634757995605, 'eval_Accuracy': 0.8864241348713399, 'eval_AUC-ROC': 0.9560709660479298, 'eval_Precision': 0.80451841570831, 'eval_Recall': 0.7921601853640271, 'eval_F1-score': 0.7971128505954014, 'eval_runtime': 3.3347, 'eval_samples_per_second': 675.93, 'eval_steps_per_second': 5.398}


In [54]:
test_results_flavor = trainer.evaluate()

print("Test Results for checkpoint 1645:", test_results_flavor)

Test Results for checkpoint 1500: {'eval_mcc_metric': 0.7893589818962506, 'eval_loss': 0.14277541637420654, 'eval_Accuracy': 0.888642413487134, 'eval_AUC-ROC': 0.970534896759912, 'eval_Precision': 0.8309968335234739, 'eval_Recall': 0.7920133444927788, 'eval_F1-score': 0.8095269496517847, 'eval_runtime': 7.7548, 'eval_samples_per_second': 290.659, 'eval_steps_per_second': 2.321}


### Result for imo6nm9x checkpoint 1645  (Best Model until now)

In [ ]:
test_results_flavor = trainer.evaluate()

print("Test Results for checkpoint 1645:", test_results_flavor) 

Test Results for checkpoint 1000: {'eval_mcc_metric': 0.7882497847517572, 'eval_loss': 0.14135079085826874, 'eval_Accuracy': 0.8873114463176575, 'eval_AUC-ROC': 0.9720325211171271, 'eval_Precision': 0.8327510094500526, 'eval_Recall': 0.8274682544764185, 'eval_F1-score': 0.8297152462405265, 'eval_runtime': 3.0135, 'eval_samples_per_second': 747.963, 'eval_steps_per_second': 5.973}


### The best model is : checkpoint 1500 of result flavor chemberta wandb for the original flavor paper model

### 10 MTR model

In [32]:
import os
from peft import PeftModel  

models_dir = "./models_MTR10_Chemberta"

for ckpt in os.listdir(models_dir):
    model_path = os.path.join(models_dir, ckpt)
    
    if os.path.isdir(model_path):
        adapter_config_path = os.path.join(model_path, "adapter_config.json")
        
        if not os.path.exists(adapter_config_path):
            print(f"⚠️ Missing 'adapter_config.json' in {model_path}")

# Evaluate each saved model

valid_checkpoints = [
    os.path.join(models_dir, ckpt)
    for ckpt in os.listdir(models_dir)
    if os.path.isdir(os.path.join(models_dir, ckpt)) and 
       os.path.exists(os.path.join(models_dir, ckpt, "adapter_config.json"))
]

print("Valid Model Checkpoints:", valid_checkpoints)

for model_path in valid_checkpoints:
    print(f"Evaluating model: {model_path}")

    adapter_model = PeftModel.from_pretrained(base_model, model_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()
    print(f"Test Results for {model_path}: {test_results_clin}")


Valid Model Checkpoints: ['./models_MTR10_Chemberta/g4esefrp', './models_MTR10_Chemberta/ejr2572h', './models_MTR10_Chemberta/ljfmymgw', './models_MTR10_Chemberta/6wz8dk9m', './models_MTR10_Chemberta/ng77zdds', './models_MTR10_Chemberta/01aq0qc9', './models_MTR10_Chemberta/qfe8vcqo', './models_MTR10_Chemberta/bc9aajo1', './models_MTR10_Chemberta/bxb0kwqm', './models_MTR10_Chemberta/wmaugze4', './models_MTR10_Chemberta/sdj2wujw', './models_MTR10_Chemberta/l3pooke3', './models_MTR10_Chemberta/6fbtk138', './models_MTR10_Chemberta/7zpn8eng', './models_MTR10_Chemberta/g9h0hhdh', './models_MTR10_Chemberta/9m90icx1', './models_MTR10_Chemberta/fh69w10f', './models_MTR10_Chemberta/izd262gb', './models_MTR10_Chemberta/1y764hkg', './models_MTR10_Chemberta/ot8tkhls']
Evaluating model: ./models_MTR10_Chemberta/g4esefrp


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_3543620/504190986.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_3543620/504190986.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = 

Test Results for ./models_MTR10_Chemberta/g4esefrp: {'eval_mcc_metric': -0.051316471359806895, 'eval_loss': 1.0326316356658936, 'eval_model_preparation_time': 0.0035, 'eval_Accuracy': 0.14640638864241348, 'eval_AUC-ROC': 0.43906379255989547, 'eval_Precision': 0.2489830873085715, 'eval_Recall': 0.18508591537359448, 'eval_F1-score': 0.14766558492259813, 'eval_runtime': 6.5404, 'eval_samples_per_second': 344.629, 'eval_steps_per_second': 10.856}
Evaluating model: ./models_MTR10_Chemberta/ejr2572h


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_3543620/504190986.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = 

...

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_3543620/504190986.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = 

Test Results for ./models_MTR10_Chemberta/izd262gb: {'eval_mcc_metric': 0.7535055121586963, 'eval_loss': 0.178395614027977, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8682342502218279, 'eval_AUC-ROC': 0.9349904085573113, 'eval_Precision': 0.7087291941627231, 'eval_Recall': 0.675850986501599, 'eval_F1-score': 0.6858847985640688, 'eval_runtime': 5.7972, 'eval_samples_per_second': 388.808, 'eval_steps_per_second': 12.247}
Evaluating model: ./models_MTR10_Chemberta/1y764hkg


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_3543620/504190986.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = 

Test Results for ./models_MTR10_Chemberta/1y764hkg: {'eval_mcc_metric': 0.7830645275961695, 'eval_loss': 0.1425733119249344, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8846495119787046, 'eval_AUC-ROC': 0.9724818401104148, 'eval_Precision': 0.8660406281416468, 'eval_Recall': 0.7554246405058954, 'eval_F1-score': 0.7920559684082156, 'eval_runtime': 6.1651, 'eval_samples_per_second': 365.604, 'eval_steps_per_second': 11.516}
Evaluating model: ./models_MTR10_Chemberta/ot8tkhls


Test Results for ./models_MTR10_Chemberta/ot8tkhls: {'eval_mcc_metric': 0.7672753355369059, 'eval_loss': 0.16462251543998718, 'eval_model_preparation_time': 0.0039, 'eval_Accuracy': 0.8762200532386868, 'eval_AUC-ROC': 0.9429835568930061, 'eval_Precision': 0.8636765181482524, 'eval_Recall': 0.6763263006647735, 'eval_F1-score': 0.7053200238365681, 'eval_runtime': 6.2716, 'eval_samples_per_second': 359.399, 'eval_steps_per_second': 11.321}


### Checkpoints of 10MTR Model

In [48]:
from peft import PeftModel  

adapter_model = PeftModel.from_pretrained(base_model, "/home/raghvendra2/Molformer_Finetuning/models_MTR10_Chemberta/qfe8vcqo/checkpoint-1500")

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")


In [49]:
trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

/tmp/ipykernel_3543620/3341849556.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


### Best model of MTR 10M model ng77zdds 1645

In [35]:
test_results_flavor = trainer.evaluate()

print("Test Results for checkpoint 1645 of ng77zdds:", test_results_flavor)

Test Results for checkpoint 1645 of ng77zdds: {'eval_mcc_metric': 0.8020882141160608, 'eval_loss': 0.1354757696390152, 'eval_model_preparation_time': 0.0055, 'eval_Accuracy': 0.8948535936113576, 'eval_AUC-ROC': 0.9712263554530509, 'eval_Precision': 0.8163329237327627, 'eval_Recall': 0.8065458720498409, 'eval_F1-score': 0.8099227080402003, 'eval_runtime': 6.2435, 'eval_samples_per_second': 361.016, 'eval_steps_per_second': 11.372}


### 2nd best

In [38]:
test_results_flavor = trainer.evaluate()

print("Test Results for checkpoint 1500 of ng77zdds:", test_results_flavor)

Test Results for checkpoint 1500 of ng77zdds: {'eval_mcc_metric': 0.7903027900598758, 'eval_loss': 0.13793590664863586, 'eval_model_preparation_time': 0.0043, 'eval_Accuracy': 0.888642413487134, 'eval_AUC-ROC': 0.9712670821306159, 'eval_Precision': 0.8297460295827033, 'eval_Recall': 0.7985800549192325, 'eval_F1-score': 0.8120801935825286, 'eval_runtime': 6.3741, 'eval_samples_per_second': 353.618, 'eval_steps_per_second': 11.139}


In [ ]:
test_results_flavor = trainer.evaluate()

print("Test Results for checkpoint 1500 of qfe8vcqo:", test_results_flavor)

Test Results for checkpoint 1500 of qfe8vcqo: {'eval_mcc_metric': 0.7895870461452488, 'eval_loss': 0.1416712999343872, 'eval_model_preparation_time': 0.0042, 'eval_Accuracy': 0.888642413487134, 'eval_AUC-ROC': 0.9652783429504371, 'eval_Precision': 0.8719392292967356, 'eval_Recall': 0.7636337052785079, 'eval_F1-score': 0.7999856428943348, 'eval_runtime': 6.3872, 'eval_samples_per_second': 352.894, 'eval_steps_per_second': 11.116}


: 

## 100 MLM model

In [44]:

import os

models_dir = "./models_Mlm_10_Chemberta"

for ckpt in os.listdir(models_dir):
    model_path = os.path.join(models_dir, ckpt)
    
    if os.path.isdir(model_path):
        adapter_config_path = os.path.join(model_path, "adapter_config.json")
        
        if not os.path.exists(adapter_config_path):
            print(f"⚠️ Missing 'adapter_config.json' in {model_path}")

# Evaluate each saved model

valid_checkpoints = [
    os.path.join(models_dir, ckpt)
    for ckpt in os.listdir(models_dir)
    if os.path.isdir(os.path.join(models_dir, ckpt)) and 
       os.path.exists(os.path.join(models_dir, ckpt, "adapter_config.json"))
]

print("Valid Model Checkpoints:", valid_checkpoints)

for model_path in valid_checkpoints:
    print(f"Evaluating model: {model_path}")

    adapter_model = PeftModel.from_pretrained(base_model, model_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()
    print(f"Test Results for {model_path}: {test_results_clin}")

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_990012/897935396.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Valid Model Checkpoints: ['./models_Mlm_10_Chemberta/fv1ilzi8', './models_Mlm_10_Chemberta/4fb63oti', './models_Mlm_10_Chemberta/gfgudjue', './models_Mlm_10_Chemberta/e374cax6', './models_Mlm_10_Chemberta/8jzpzyyg', './models_Mlm_10_Chemberta/ao6l1ar5', './models_Mlm_10_Chemberta/29mn1i36', './models_Mlm_10_Chemberta/v1e93rjp', './models_Mlm_10_Chemberta/4lskz3k4', './models_Mlm_10_Chemberta/6ocfxa7u']
Evaluating model: ./models_Mlm_10_Chemberta/fv1ilzi8


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_990012/897935396.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = C

Test Results for ./models_Mlm_10_Chemberta/fv1ilzi8: {'eval_mcc_metric': 0.0704170504547604, 'eval_loss': 1.0308160781860352, 'eval_model_preparation_time': 0.0033, 'eval_Accuracy': 0.2067435669920142, 'eval_AUC-ROC': 0.5201085713307327, 'eval_Precision': 0.2973564404255674, 'eval_Recall': 0.22460865555170856, 'eval_F1-score': 0.15843567515925416, 'eval_runtime': 3.6606, 'eval_samples_per_second': 615.744, 'eval_steps_per_second': 19.396}
Evaluating model: ./models_Mlm_10_Chemberta/4fb63oti


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_990012/897935396.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = C

...

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_990012/897935396.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = C

Test Results for ./models_Mlm_10_Chemberta/v1e93rjp: {'eval_mcc_metric': 0.7307316602577645, 'eval_loss': 0.19169306755065918, 'eval_model_preparation_time': 0.0037, 'eval_Accuracy': 0.8589174800354925, 'eval_AUC-ROC': 0.9121644495936069, 'eval_Precision': 0.8309833144548868, 'eval_Recall': 0.6498413016992705, 'eval_F1-score': 0.6802427550062317, 'eval_runtime': 3.5312, 'eval_samples_per_second': 638.311, 'eval_steps_per_second': 20.107}
Evaluating model: ./models_Mlm_10_Chemberta/4lskz3k4


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_990012/897935396.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = C

Test Results for ./models_Mlm_10_Chemberta/4lskz3k4: {'eval_mcc_metric': 0.7349980888326919, 'eval_loss': 0.19541019201278687, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8593611357586513, 'eval_AUC-ROC': 0.9138323763599596, 'eval_Precision': 0.7295496767165879, 'eval_Recall': 0.6574871971495032, 'eval_F1-score': 0.6758098650437983, 'eval_runtime': 3.6219, 'eval_samples_per_second': 622.322, 'eval_steps_per_second': 19.603}
Evaluating model: ./models_Mlm_10_Chemberta/6ocfxa7u


Test Results for ./models_Mlm_10_Chemberta/6ocfxa7u: {'eval_mcc_metric': 0.7414589304407849, 'eval_loss': 0.1874830275774002, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8633540372670807, 'eval_AUC-ROC': 0.9239020585135893, 'eval_Precision': 0.833281359845302, 'eval_Recall': 0.661965819696989, 'eval_F1-score': 0.6879138746837524, 'eval_runtime': 3.8368, 'eval_samples_per_second': 587.463, 'eval_steps_per_second': 18.505}


### Checking for all 5M MTR models

In [30]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-5M-MTR",
    num_labels=5,
    problem_type="single_label_classification",    
    trust_remote_code=True,
    
)

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-5M-MTR",trust_remote_code=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-5M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from peft import PeftModel  
import os

models_dir = "./models_Mtr_5_Chemberta"

for ckpt in os.listdir(models_dir):
    model_path = os.path.join(models_dir, ckpt)
    
    if os.path.isdir(model_path):
        adapter_config_path = os.path.join(model_path, "adapter_config.json")
        
        if not os.path.exists(adapter_config_path):
            print(f"⚠️ Missing 'adapter_config.json' in {model_path}")

# Evaluate each saved model

valid_checkpoints = [
    os.path.join(models_dir, ckpt)
    for ckpt in os.listdir(models_dir)
    if os.path.isdir(os.path.join(models_dir, ckpt)) and 
       os.path.exists(os.path.join(models_dir, ckpt, "adapter_config.json"))
]

print("Valid Model Checkpoints:", valid_checkpoints)

for model_path in valid_checkpoints:
    print(f"Evaluating model: {model_path}")

    adapter_model = PeftModel.from_pretrained(base_model, model_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()
    print(f"Test Results for {model_path}: {test_results_clin}")

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1992331/994277692.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


⚠️ Missing 'adapter_config.json' in ./models_Mtr_5_Chemberta/84cefzee
Valid Model Checkpoints: ['./models_Mtr_5_Chemberta/eeuaszyg', './models_Mtr_5_Chemberta/wvxxdsok', './models_Mtr_5_Chemberta/hfjmj8af', './models_Mtr_5_Chemberta/i5zckto9', './models_Mtr_5_Chemberta/24p9zgdn', './models_Mtr_5_Chemberta/io1c4ug5', './models_Mtr_5_Chemberta/nq42s50m', './models_Mtr_5_Chemberta/lfvffslg', './models_Mtr_5_Chemberta/4p2kl0le', './models_Mtr_5_Chemberta/ret7ydyc', './models_Mtr_5_Chemberta/v773pzco', './models_Mtr_5_Chemberta/qlr8qiyn', './models_Mtr_5_Chemberta/7lx28l2q', './models_Mtr_5_Chemberta/ndgq1b2l', './models_Mtr_5_Chemberta/0n7q4g0x', './models_Mtr_5_Chemberta/s53s98co', './models_Mtr_5_Chemberta/r7mxurpz', './models_Mtr_5_Chemberta/7fpw74gq', './models_Mtr_5_Chemberta/1uc5cs8k', './models_Mtr_5_Chemberta/qdtwt30e', './models_Mtr_5_Chemberta/sj3yydqn', './models_Mtr_5_Chemberta/cgefgnd3', './models_Mtr_5_Chemberta/5x17bw96', './models_Mtr_5_Chemberta/j92alz30']
Evaluating model

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1992331/994277692.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = 

Test Results for ./models_Mtr_5_Chemberta/eeuaszyg: {'eval_mcc_metric': 0.0651055566688356, 'eval_loss': 1.035671353340149, 'eval_model_preparation_time': 0.0034, 'eval_Accuracy': 0.2684117125110914, 'eval_AUC-ROC': 0.6048832895929168, 'eval_Precision': 0.26998547654108057, 'eval_Recall': 0.2531751674551164, 'eval_F1-score': 0.1977021888255427, 'eval_runtime': 3.5781, 'eval_samples_per_second': 629.947, 'eval_steps_per_second': 19.843}
Evaluating model: ./models_Mtr_5_Chemberta/wvxxdsok


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1992331/994277692.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = 

...

Test Results for ./models_Mtr_5_Chemberta/cgefgnd3: {'eval_mcc_metric': 0.7722590521446594, 'eval_loss': 0.14912612736225128, 'eval_model_preparation_time': 0.0039, 'eval_Accuracy': 0.8802129547471162, 'eval_AUC-ROC': 0.9589468942440134, 'eval_Precision': 0.76052944188729, 'eval_Recall': 0.7119211833907053, 'eval_F1-score': 0.7318655197216989, 'eval_runtime': 3.658, 'eval_samples_per_second': 616.179, 'eval_steps_per_second': 19.409}
Evaluating model: ./models_Mtr_5_Chemberta/5x17bw96


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weigh

Test Results for ./models_Mtr_5_Chemberta/5x17bw96: {'eval_mcc_metric': 0.732940156486814, 'eval_loss': 0.1951296329498291, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8589174800354925, 'eval_AUC-ROC': 0.9232400794964539, 'eval_Precision': 0.6351294160479245, 'eval_Recall': 0.6204615992745854, 'eval_F1-score': 0.6239080423384326, 'eval_runtime': 3.5885, 'eval_samples_per_second': 628.116, 'eval_steps_per_second': 19.785}
Evaluating model: ./models_Mtr_5_Chemberta/j92alz30


Test Results for ./models_Mtr_5_Chemberta/j92alz30: {'eval_mcc_metric': 0.5544837921085852, 'eval_loss': 0.31705886125564575, 'eval_model_preparation_time': 0.0039, 'eval_Accuracy': 0.782608695652174, 'eval_AUC-ROC': 0.8121271977935832, 'eval_Precision': 0.586801825020637, 'eval_Recall': 0.46280637664456503, 'eval_F1-score': 0.49256044259750764, 'eval_runtime': 3.5959, 'eval_samples_per_second': 626.832, 'eval_steps_per_second': 19.745}


/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Checking for checkpoints of 5M MTR 

In [96]:
from peft import PeftModel  

adapter_model = PeftModel.from_pretrained(base_model, "/home/raghvendra2/Molformer_Finetuning/models_Mtr_5_Chemberta/ndgq1b2l/checkpoint-1645")

/home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")


In [97]:
trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

/tmp/ipykernel_1992331/3341849556.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [98]:
test_results_flavor = trainer.evaluate()

print("Test Results for checkpoint 1645 of sj3yydqn:", test_results_flavor)

Test Results for checkpoint 1645 of sj3yydqn: {'eval_mcc_metric': 0.7810350014174989, 'eval_loss': 0.1457151621580124, 'eval_model_preparation_time': 0.0043, 'eval_Accuracy': 0.8846495119787046, 'eval_AUC-ROC': 0.9527411756627775, 'eval_Precision': 0.7988920909771744, 'eval_Recall': 0.7210630577180025, 'eval_F1-score': 0.7486025273539199, 'eval_runtime': 4.7489, 'eval_samples_per_second': 474.638, 'eval_steps_per_second': 14.951}


### 10MLM Model performance

In [75]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MTR",
    num_labels=5,
    problem_type="single_label_classification",    
    trust_remote_code=True,
    
)

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR",trust_remote_code=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [76]:

import os



for ckpt in os.listdir(models_dir):import os
from peft import PeftModel

models_dir = "./models_MTR100_Chemberta"

def find_all_checkpoints(base_dir):
    all_checkpoints = []
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path):
            # Now look inside this folder for checkpoint-* subdirs
            for subfolder in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.isdir(subfolder_path) and subfolder.startswith("checkpoint-"):
                    # Check for adapter_config.json
                    if os.path.exists(os.path.join(subfolder_path, "adapter_config.json")):
                        all_checkpoints.append(subfolder_path)
    return all_checkpoints

valid_checkpoints = find_all_checkpoints(models_dir)
print("🧠 Valid nested checkpoints found:", valid_checkpoints)

for checkpoint_path in valid_checkpoints:
    checkpoint_name = os.path.basename(checkpoint_path)
    parent_folder = os.path.basename(os.path.dirname(checkpoint_path))

    print(f"\n🔍 Evaluating model: {parent_folder}/{checkpoint_name}")

    adapter_model = PeftModel.from_pretrained(base_model, checkpoint_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()

    print(f"📌 Test Results for {parent_folder}/{checkpoint_name}: {test_results_clin}")

    model_path = os.path.join(models_dir, ckpt)
    
    if os.path.isdir(model_path):
        adapter_config_path = os.path.join(model_path, "adapter_config.json")
        
        if not os.path.exists(adapter_config_path):
            print(f"⚠️ Missing 'adapter_config.json' in {model_path}")

# Evaluate each saved model

valid_checkpoints = [
    os.path.join(models_dir, ckpt)
    for ckpt in os.listdir(models_dir)
    if os.path.isdir(os.path.join(models_dir, ckpt)) and 
       os.path.exists(os.path.join(models_dir, ckpt, "adapter_config.json"))
]

print("Valid Model Checkpoints:", valid_checkpoints)

for model_path in valid_checkpoints:
    print(f"Evaluating model: {model_path}")

    adapter_model = PeftModel.from_pretrained(base_model, model_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()
    print(f"Test Results for {model_path}: {test_results_clin}")

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3224754749.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🧠 Valid nested checkpoints found: ['./models_MTR100_Chemberta/ujye8pur/checkpoint-500', './models_MTR100_Chemberta/glzr8jim/checkpoint-1500', './models_MTR100_Chemberta/glzr8jim/checkpoint-1000', './models_MTR100_Chemberta/glzr8jim/checkpoint-500', './models_MTR100_Chemberta/glzr8jim/checkpoint-1645', './models_MTR100_Chemberta/y6ioij00/checkpoint-1500', './models_MTR100_Chemberta/y6ioij00/checkpoint-1000', './models_MTR100_Chemberta/y6ioij00/checkpoint-500', './models_MTR100_Chemberta/y6ioij00/checkpoint-1645', './models_MTR100_Chemberta/5k0m1ybg/checkpoint-1500', './models_MTR100_Chemberta/5k0m1ybg/checkpoint-1000', './models_MTR100_Chemberta/5k0m1ybg/checkpoint-500', './models_MTR100_Chemberta/5k0m1ybg/checkpoint-1645', './models_MTR100_Chemberta/imo6nm9x/checkpoint-1500', './models_MTR100_Chemberta/imo6nm9x/checkpoint-1000', './models_MTR100_Chemberta/imo6nm9x/checkpoint-500', './models_MTR100_Chemberta/imo6nm9x/checkpoint-1645', './models_MTR100_Chemberta/to3uvb1h/checkpoint-1500'

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3224754749.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

📌 Test Results for ujye8pur/checkpoint-500: {'eval_mcc_metric': 0.19421155282376865, 'eval_loss': 0.9777860045433044, 'eval_Accuracy': 0.48535936113575867, 'eval_AUC-ROC': 0.6205852690220697, 'eval_Precision': 0.2693402384519366, 'eval_Recall': 0.32627241350001607, 'eval_F1-score': 0.236805625235501, 'eval_runtime': 2.9214, 'eval_samples_per_second': 771.552, 'eval_steps_per_second': 6.161}

🔍 Evaluating model: glzr8jim/checkpoint-1500


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3224754749.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

...

Test Results for ./models_MTR100_Chemberta/imo6nm9x: {'eval_mcc_metric': 0.7818169758422309, 'eval_loss': 0.1437530368566513, 'eval_Accuracy': 0.8850931677018633, 'eval_AUC-ROC': 0.9721605705286832, 'eval_Precision': 0.8250201990356656, 'eval_Recall': 0.7875592578624443, 'eval_F1-score': 0.8045247463831717, 'eval_runtime': 2.9361, 'eval_samples_per_second': 767.688, 'eval_steps_per_second': 6.131}
Evaluating model: ./models_MTR100_Chemberta/to3uvb1h


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3224754749.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

Test Results for ./models_MTR100_Chemberta/to3uvb1h: {'eval_mcc_metric': 0.777782885687969, 'eval_loss': 0.1434350311756134, 'eval_Accuracy': 0.883318544809228, 'eval_AUC-ROC': 0.9714154098726826, 'eval_Precision': 0.8247383778991484, 'eval_Recall': 0.7833793596443682, 'eval_F1-score': 0.8022087198646425, 'eval_runtime': 2.9953, 'eval_samples_per_second': 752.507, 'eval_steps_per_second': 6.009}
Evaluating model: ./models_MTR100_Chemberta/oggmqthu


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3224754749.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

Test Results for ./models_MTR100_Chemberta/oggmqthu: {'eval_mcc_metric': 0.7667172633940723, 'eval_loss': 0.14829595386981964, 'eval_Accuracy': 0.8757763975155279, 'eval_AUC-ROC': 0.9531141891298681, 'eval_Precision': 0.7546311912638144, 'eval_Recall': 0.746212206833805, 'eval_F1-score': 0.7496316603246458, 'eval_runtime': 3.3239, 'eval_samples_per_second': 678.117, 'eval_steps_per_second': 5.415}


## 77M MLM Model

In [22]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MLM",
    num_labels=5,
    problem_type="single_label_classification",    
    trust_remote_code=True,
    
)

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM",trust_remote_code=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
training_args = TrainingArguments(
    output_dir="./test_results_flavor",
    per_device_eval_batch_size=32,
    report_to="none",  # Disable logging to W&B for test
    

)


In [ ]:
import os
from peft import PeftModel

models_dir = "./models_BBBP_chemberta_focal_loss"

def find_all_checkpoints(base_dir):
    all_checkpoints = []
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path):
            for subfolder in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.isdir(subfolder_path) and subfolder.startswith("checkpoint-"):
                    if os.path.exists(os.path.join(subfolder_path, "adapter_config.json")):
                        all_checkpoints.append(subfolder_path)
    return all_checkpoints

valid_checkpoints = find_all_checkpoints(models_dir)
print("🧠 Valid nested checkpoints found:", valid_checkpoints)

for checkpoint_path in valid_checkpoints:
    checkpoint_name = os.path.basename(checkpoint_path)
    parent_folder = os.path.basename(os.path.dirname(checkpoint_path))

    print(f"\n🔍 Evaluating model: {parent_folder}/{checkpoint_name}")

    adapter_model = PeftModel.from_pretrained(base_model, checkpoint_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()

    auc_score = test_results_clin.get("eval/AUC_ROC", 0)
    if auc_score > 0.95:
        print(f"✅ AUC_ROC > 0.95 for {parent_folder}/{checkpoint_name}")
        print(f"📌 Test Results: {test_results_clin}")
    else:
        print(f"❌ Skipping {parent_folder}/{checkpoint_name} (AUC_ROC = {auc_score:.4f})")


🧠 Valid nested checkpoints found: ['./models_Mlm_100_Chemberta/xutmnj12/checkpoint-1500', './models_Mlm_100_Chemberta/xutmnj12/checkpoint-1000', './models_Mlm_100_Chemberta/xutmnj12/checkpoint-500', './models_Mlm_100_Chemberta/xutmnj12/checkpoint-1645', './models_Mlm_100_Chemberta/sz247wpr/checkpoint-658', './models_Mlm_100_Chemberta/sz247wpr/checkpoint-500', './models_Mlm_100_Chemberta/ikd7q7fk/checkpoint-658', './models_Mlm_100_Chemberta/ikd7q7fk/checkpoint-500', './models_Mlm_100_Chemberta/kovlgv1s/checkpoint-1500', './models_Mlm_100_Chemberta/kovlgv1s/checkpoint-1000', './models_Mlm_100_Chemberta/kovlgv1s/checkpoint-500', './models_Mlm_100_Chemberta/kovlgv1s/checkpoint-1645', './models_Mlm_100_Chemberta/k0sfrn0p/checkpoint-1500', './models_Mlm_100_Chemberta/k0sfrn0p/checkpoint-1000', './models_Mlm_100_Chemberta/k0sfrn0p/checkpoint-500', './models_Mlm_100_Chemberta/k0sfrn0p/checkpoint-1645', './models_Mlm_100_Chemberta/0yvjid5t/checkpoint-658', './models_Mlm_100_Chemberta/0yvjid5t/c

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/179427769.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.

📌 Test Results for xutmnj12/checkpoint-1500: {'eval_mcc_metric': 0.7515927803916811, 'eval_loss': 0.19172461330890656, 'eval_Accuracy': 0.8700088731144632, 'eval_AUC-ROC': 0.9528106082436698, 'eval_Precision': 0.7540780668366062, 'eval_Recall': 0.726382307268234, 'eval_F1-score': 0.7377056675422857, 'eval_runtime': 3.1689, 'eval_samples_per_second': 711.282, 'eval_steps_per_second': 5.68}

🔍 Evaluating model: xutmnj12/checkpoint-1000


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/179427769.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.

📌 Test Results for xutmnj12/checkpoint-1000: {'eval_mcc_metric': 0.7381556177411126, 'eval_loss': 0.20143990218639374, 'eval_Accuracy': 0.8655723158828749, 'eval_AUC-ROC': 0.9462037248705334, 'eval_Precision': 0.8562507949004768, 'eval_Recall': 0.6418617688488828, 'eval_F1-score': 0.6859070010245756, 'eval_runtime': 2.9426, 'eval_samples_per_second': 765.981, 'eval_steps_per_second': 6.117}

🔍 Evaluating model: xutmnj12/checkpoint-500


...

📌 Test Results for 532smrg2/checkpoint-500: {'eval_mcc_metric': 0.734268880398551, 'eval_loss': 0.20883527398109436, 'eval_Accuracy': 0.8589174800354925, 'eval_AUC-ROC': 0.9440477414069232, 'eval_Precision': 0.7009575272273703, 'eval_Recall': 0.7516299954479507, 'eval_F1-score': 0.7085234353197551, 'eval_runtime': 2.9781, 'eval_samples_per_second': 756.847, 'eval_steps_per_second': 6.044}

🔍 Evaluating model: gbyjoy6e/checkpoint-658


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/179427769.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out

📌 Test Results for gbyjoy6e/checkpoint-658: {'eval_mcc_metric': 0.7242799948767075, 'eval_loss': 0.2091398537158966, 'eval_Accuracy': 0.8553682342502218, 'eval_AUC-ROC': 0.9401006451281212, 'eval_Precision': 0.6406431158116671, 'eval_Recall': 0.6096868249240435, 'eval_F1-score': 0.6204827610222969, 'eval_runtime': 2.935, 'eval_samples_per_second': 767.966, 'eval_steps_per_second': 6.133}

🔍 Evaluating model: gbyjoy6e/checkpoint-500


📌 Test Results for gbyjoy6e/checkpoint-500: {'eval_mcc_metric': 0.7299963331063521, 'eval_loss': 0.2147645652294159, 'eval_Accuracy': 0.8553682342502218, 'eval_AUC-ROC': 0.8971433581892038, 'eval_Precision': 0.7331086934361826, 'eval_Recall': 0.6519809960114861, 'eval_F1-score': 0.6698237032430857, 'eval_runtime': 2.9354, 'eval_samples_per_second': 767.86, 'eval_steps_per_second': 6.132}


In [43]:
from peft import PeftModel  
import os

models_dir = "./models_Mlm_100_Chemberta"

for ckpt in os.listdir(models_dir):
    model_path = os.path.join(models_dir, ckpt)
    
    if os.path.isdir(model_path):
        adapter_config_path = os.path.join(model_path, "adapter_config.json")
        
        if not os.path.exists(adapter_config_path):
            print(f"⚠️ Missing 'adapter_config.json' in {model_path}")

# Evaluate each saved model

valid_checkpoints = [
    os.path.join(models_dir, ckpt)
    for ckpt in os.listdir(models_dir)
    if os.path.isdir(os.path.join(models_dir, ckpt)) and 
       os.path.exists(os.path.join(models_dir, ckpt, "adapter_config.json"))
]

print("Valid Model Checkpoints:", valid_checkpoints)

for model_path in valid_checkpoints:
    print(f"Evaluating model: {model_path}")
    checkpoint_name = os.path.basename(model_path)
    

    model_files = os.listdir(model_path)

     # Identify key model files (e.g., adapter weights)
    adapter_files = [f for f in model_files if f.endswith(".bin")]
    adapter_file_used = adapter_files[0] if adapter_files else "❌ No .bin file found"

    print(f"\n🔍 Evaluating model folder: {checkpoint_name}")
    print(f"📦 Adapter file being loaded: {adapter_file_used}")

    adapter_model = PeftModel.from_pretrained(base_model, model_path)
    adapter_model.eval()

    trainer = CustomTrainer(
        model=adapter_model,
        args=training_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    test_results_clin = trainer.evaluate()
    print(f"Test Results for {model_path}: {test_results_clin}")

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3232351798.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

Valid Model Checkpoints: ['./models_Mlm_100_Chemberta/xutmnj12', './models_Mlm_100_Chemberta/sz247wpr', './models_Mlm_100_Chemberta/ikd7q7fk', './models_Mlm_100_Chemberta/kovlgv1s', './models_Mlm_100_Chemberta/k0sfrn0p', './models_Mlm_100_Chemberta/0yvjid5t', './models_Mlm_100_Chemberta/d1wrlvdu', './models_Mlm_100_Chemberta/d6a28ks6', './models_Mlm_100_Chemberta/3fdjf7s7', './models_Mlm_100_Chemberta/6a8xs6i0', './models_Mlm_100_Chemberta/a99qvoss', './models_Mlm_100_Chemberta/q6ab3hnx', './models_Mlm_100_Chemberta/jbxogsm2', './models_Mlm_100_Chemberta/gi5kecmi', './models_Mlm_100_Chemberta/u9d6q2lo', './models_Mlm_100_Chemberta/wt5mgigm', './models_Mlm_100_Chemberta/532smrg2', './models_Mlm_100_Chemberta/gbyjoy6e']
Evaluating model: ./models_Mlm_100_Chemberta/xutmnj12

🔍 Evaluating model folder: xutmnj12
📦 Adapter file being loaded: training_args.bin


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3232351798.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

Test Results for ./models_Mlm_100_Chemberta/xutmnj12: {'eval_mcc_metric': 0.7521464915364129, 'eval_loss': 0.19328723847866058, 'eval_Accuracy': 0.870452528837622, 'eval_AUC-ROC': 0.9531784124231235, 'eval_Precision': 0.7572555794128578, 'eval_Recall': 0.7250657272294483, 'eval_F1-score': 0.7380485089709764, 'eval_runtime': 2.939, 'eval_samples_per_second': 766.931, 'eval_steps_per_second': 6.125}
Evaluating model: ./models_Mlm_100_Chemberta/sz247wpr

🔍 Evaluating model folder: sz247wpr
📦 Adapter file being loaded: training_args.bin


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3232351798.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

...

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3232351798.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

Test Results for ./models_Mlm_100_Chemberta/wt5mgigm: {'eval_mcc_metric': 0.7556197394027053, 'eval_loss': 0.18587109446525574, 'eval_Accuracy': 0.8708961845607809, 'eval_AUC-ROC': 0.9508821803277767, 'eval_Precision': 0.724016556454012, 'eval_Recall': 0.6982829700966494, 'eval_F1-score': 0.7098701024520846, 'eval_runtime': 3.237, 'eval_samples_per_second': 696.318, 'eval_steps_per_second': 5.561}
Evaluating model: ./models_Mlm_100_Chemberta/532smrg2

🔍 Evaluating model folder: 532smrg2
📦 Adapter file being loaded: training_args.bin


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1701486/3232351798.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead

Test Results for ./models_Mlm_100_Chemberta/532smrg2: {'eval_mcc_metric': 0.7463711208398681, 'eval_loss': 0.199150949716568, 'eval_Accuracy': 0.8655723158828749, 'eval_AUC-ROC': 0.9486051174719886, 'eval_Precision': 0.7437086216358899, 'eval_Recall': 0.7269904582435949, 'eval_F1-score': 0.7321706546551738, 'eval_runtime': 3.2236, 'eval_samples_per_second': 699.208, 'eval_steps_per_second': 5.584}
Evaluating model: ./models_Mlm_100_Chemberta/gbyjoy6e

🔍 Evaluating model folder: gbyjoy6e
📦 Adapter file being loaded: training_args.bin


Test Results for ./models_Mlm_100_Chemberta/gbyjoy6e: {'eval_mcc_metric': 0.7299963331063521, 'eval_loss': 0.2147645652294159, 'eval_Accuracy': 0.8553682342502218, 'eval_AUC-ROC': 0.8971433581892038, 'eval_Precision': 0.7331086934361826, 'eval_Recall': 0.6519809960114861, 'eval_F1-score': 0.6698237032430857, 'eval_runtime': 2.9481, 'eval_samples_per_second': 764.549, 'eval_steps_per_second': 6.106}


## Best of all Models

### Best of 77M MLM Model: 

'eval_mcc_metric': 0.743591092208974, 'eval_loss': 0.19872386753559113, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8646850044365573, 'eval_AUC-ROC': 0.9008098793247992, 'eval_Precision': 0.8414375896683746, 'eval_Recall': 0.6564035372933563, 'eval_F1-score': 0.6835677143059256,

### Best of 10M MLM Model: 

{'eval_mcc_metric': 0.7783196473274908, 'eval_loss': 0.15798771381378174, 'eval_Accuracy': 0.883318544809228, 'eval_AUC-ROC': 0.9487068975033708, 'eval_Precision': 0.7174260972868147, 'eval_Recall': 0.7161449075205971, 'eval_F1-score': 0.714283627299657}

### Best of 77M MTR

'eval_mcc_metric': 0.7882497847517572, 'eval_loss': 0.14135079085826874, 'eval_Accuracy': 0.8873114463176575, 'eval_AUC-ROC': 0.9720325211171271, 'eval_Precision': 0.8327510094500526, 'eval_Recall': 0.8274682544764185, 'eval_F1-score': 0.8297152462405265


### Best of 10M MTR
'eval_mcc_metric': 0.8020882141160608, 'eval_loss': 0.1354757696390152, 'eval_model_preparation_time': 0.0055, 'eval_Accuracy': 0.8948535936113576, 'eval_AUC-ROC': 0.9712263554530509, 'eval_Precision': 0.8163329237327627, 'eval_Recall': 0.8065458720498409, 'eval_F1-score': 0.8099227080402003,

'eval_mcc_metric': 0.7903027900598758, 'eval_loss': 0.13793590664863586, 'eval_model_preparation_time': 0.0043, 'eval_Accuracy': 0.888642413487134, 'eval_AUC-ROC': 0.9712670821306159, 'eval_Precision': 0.8297460295827033, 'eval_Recall': 0.7985800549192325, 'eval_F1-score': 0.8120801935825286,

### Best of 5M MTR

'eval_mcc_metric': 0.7922832370919513, 'eval_loss': 0.13667497038841248, 'eval_model_preparation_time': 0.0039, 'eval_Accuracy': 0.8895297249334516, 'eval_AUC-ROC': 0.958335401296123, 'eval_Precision': 0.767476398219038, 'eval_Recall': 0.7331228786790375, 'eval_F1-score': 0.7469094345185043

###  With the model chemberta_v1_ the performance was worse relatively

## Load and Merge the best model

In [62]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    'DeepChem/ChemBERTa-10M-MLM',
    num_labels=5,
    problem_type="single_label_classification",    
    trust_remote_code=True,
    
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
from peft import PeftModel  

adapter_model = PeftModel.from_pretrained(base_model, "/home/raghvendra2/Molformer_Finetuning/models_Mlm_10_Chemberta/ad5xf1ya/checkpoint-1500")

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")


In [64]:
final_model_chemberta_10M_MLM= adapter_model.merge_and_unload()

### Save model to Chemberta finetuned model lora 100M MTR

In [6]:
save_path = "/home/raghvendra2/Molformer_Finetuning/chemberta_final_model_lora_100M_MTR"

final_model_chemberta_77M_MTR.save_pretrained(save_path)



### Save Model for 77M MLM

In [49]:
save_path = "/home/raghvendra2/Molformer_Finetuning/chemberta_final_model_lora_77M_MLM"

final_model_chemberta_77M_MLM.save_pretrained(save_path) 

### save Model for 10M MLM

In [65]:
save_path = "/home/raghvendra2/Molformer_Finetuning/chemberta_final_model_lora_10M_MLM"

final_model_chemberta_10M_MLM.save_pretrained(save_path)

### Save Model for 10M MTR

In [ ]:
save_path = "/home/raghvendra2/Molformer_Finetuning/chemberta_final_model_lora_10M_MTR"

final_model_chemberta_10M_MTR.save_pretrained(save_path)

### Save model for 5M MTR

In [21]:
save_path = "/home/raghvendra2/Molformer_Finetuning/chemberta_final_model_lora_5M_MTR"

final_model_chemberta_5M_MTR.save_pretrained(save_path)

In [7]:
# Count total parameters
total_params = sum(p.numel() for p in final_model_chemberta_77M_MTR.parameters())
# Count trainable parameters
trainable_params = sum(p.numel() for p in final_model_chemberta_77M_MTR.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")

Total Parameters: 3,429,365
Trainable Parameters: 149,765


In [39]:
total_params = sum(p.numel() for p in base_model.parameters())
print(total_params)

83454725
